In [13]:
from evaluation import *
from utils_donnee import *
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import unicodedata

In [14]:
preprocessor = lambda text: stem(remove_tags(uppercase_suppression(chiffre_suppression(ponc_suppression(text)))))
#preprocessing stopwords
stemmer = PorterStemmer()
english_stop_words = stopwords.words('english')
preprocessed_eng_stop_words = [unicodedata.normalize('NFKD', stemmer.stem(word)).encode('ascii', 'ignore').decode('utf-8') for word in english_stop_words]

In [15]:


vect_params = {'lowercase':True,'strip_accents':'ascii','stop_words':preprocessed_eng_stop_words}

model_params = {'random_state':0}
predictions = prediction_generator(preprocessor=preprocessor,
                                   vectorizer=CountVectorizer,
                                   vect_params=vect_params,
                                   model=LinearSVC,
                                   model_params=model_params,
                                   save=True)


25000


C:\Users\PC\miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[0 1 0 0]


C:\Users\PC\miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [12]:
import numpy as np
import matplotlib.pyplot as plt
import codecs
import re
import os.path
import string
import nltk
from nltk.stem.snowball import FrenchStemmer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import Counter
from utils_donnee import *
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

def save_pred(pred):
    """ Sauvegarder les prédictions dans un fichier
    Entrée : Array (de M et C)
    Sortie : None + file dans predictions
    """
    directory = "./predictions"
    index = len(os.listdir(directory))+1 #nb de fichiers + 1 = l'indice du nouveau fichier
    filename = os.path.join(directory,f'test_{index}.txt')
    np.savetxt(filename, pred, fmt="%s")


def prediction_generator(preprocessor,vectorizer,vect_params,model,model_params,save=True):
    """Faire une prediction sur le fichier selon preprocessor, vectorizer, model donnés.
    Entrée : file : nom du fichier
             preprocessor : preprocessor pour nettoyer les données
             vectorizer : vectorizer à utiliser
             vect_params : parametres de vectorizer à utiliser
             model : modèle à utiliser
             model_params : paramètres du modèle à utiliser
             save : si True, sauvegarder le résultat dans un fichier
    Sortie : Array de M et C + fichier contenant ce vecteur 
    """

    # chargement des données train 
    alltxts_train,labs_train = load_movies("./datasets/movies/movies1000/")
    # chargement des données test
    alltxts_test = load_movies_test("./datasets/movies/testSentiment.txt")

    # Vectorization
    vec = vectorizer(preprocessor=preprocessor,**vect_params)
    txts_train = vec.fit_transform(alltxts_train)

    # Training
    [X_train, X_test, y_train, y_test]  = train_test_split(txts_train, labs_train, test_size=0.2, random_state=10, shuffle=True)

    # Modélisation 
    mod = model(**model_params)
    mod.fit(txts_train,labs_train)

    # Prédiction
    pred_train = mod.predict(X_train)
    pred_test = mod.predict(alltxts_test)

    pred = np.where(pred_test == 0, "N", "P")
    save_pred(pred_test)
    if save==True:
        save_pred(pred_test)

    return pred

preprocessor = lambda text: stem(remove_tags(chiffre_suppression(ponc_suppression(text))))
vect_params = {'lowercase':True,'strip_accents':'ascii','stop_words':preprocessed_eng_stop_words}

model_params = {'random_state':0}
predictions = prediction_generator(preprocessor=preprocessor,
                                   vectorizer=CountVectorizer,
                                   vect_params=vect_params,
                                   model=LinearSVC,
                                   model_params=model_params,
                                   save=True)

25000


C:\Users\PC\miniconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\PC\miniconda3\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=["Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.\n"
 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive 